In [ ]:
import pandas as pd
import scanpy as sc
import decoupler as dc
import liana as li
import os
import anndata as ad
import numpy as np
import plotnine as p9
import seaborn as sns

import mudata as md
from mudata import MuData
from itertools import product

sns.set()

In [ ]:
# Automatically re-load wrapper functions after an update
# Find details here: https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [ ]:
sc.logging.print_versions()
# sc.set_figure_params(facecolor="white", figsize=(6, 6))
sc.settings.verbosity = 3

In [ ]:
root_path = os.getcwd()
results_folder = os.path.join(root_path, 'results/deconvolution_bbknn/celltype5_updated_v1')
ref_run_name =  os.path.join(results_folder, 'reference_signatures') 
run_name = os.path.join(results_folder, 'cell2location_map') 

In [ ]:
adata_file = f"{run_name}/sp.h5ad"
adata = sc.read_h5ad(adata_file)

In [ ]:
def select_slide(adata, s, batch_key="sample"):
    r"""This function selects the data for one slide from the spatial anndata object.

    :param adata: Anndata object with multiple spatial experiments
    :param s: name of selected experiment
    :param batch_key: column in adata.obs listing experiment name for each location
    """

    slide = adata[adata.obs[batch_key].isin([s]), :].copy()
    s_keys = list(slide.uns["spatial"].keys())
    s_spatial = np.array(s_keys)[[s in k for k in s_keys]][0]

    slide.uns["spatial"] = {s_spatial: slide.uns["spatial"][s_spatial]}

    return slide

In [ ]:
sample_names = adata.obs['readout_id'].unique().tolist()

In [ ]:
sample_names

In [ ]:
for current_sample in sample_names: 
    current_adata = select_slide(adata, s = current_sample, batch_key='readout_id')

    sc.pp.normalize_total(current_adata, inplace=True, target_sum=1e4)
    sc.pp.log1p(current_adata)
    sc.pp.filter_genes(current_adata, min_cells=10)

    sc.pl.spatial(current_adata, color = ["Ltbr", 'Ltb'], ncols=2, size=1.4, cmap="coolwarm")

In [ ]:
li.mt.bivariate.show_functions()

In [ ]:
li.resource.show_resources()

In [ ]:
df_interactions = li.resource.select_resource(resource_name='mouseconsensus')

In [ ]:
df_interactions = pd.concat([df_interactions, pd.DataFrame(data={"ligand": ["Ltb"], "receptor": ["Ltbr"]})], ignore_index=True)

In [ ]:
df_interactions

In [ ]:
plot, _ = li.ut.query_bandwidth(coordinates=current_adata.obsm['spatial'], start=0, end=500, interval_n=20)
plot

In [ ]:
target_interactions = ['Ltb^Ltbr']
all_interactions = []

for current_sample in sample_names: 
    print(f"Processing sample: {current_sample}")
    
    # ... (Your existing code for select_slide, spatial_neighbors, etc.) ...
    current_adata = select_slide(adata, s=current_sample, batch_key='readout_id')
    current_condition = current_adata.obs['CONDITION'].unique()[0]
    li.ut.spatial_neighbors(current_adata, bandwidth=200, cutoff=0.1, kernel='gaussian', set_diag=True)
    li.pl.connectivity(current_adata, idx=0, size=1.3, figure_size=(6, 5))

    # Calculate bivariate interactions
    lrdata = li.mt.bivariate(
        current_adata,
        resource=df_interactions,
        local_name='cosine',
        global_name="morans",
        n_perms=100,
        mask_negatives=False,
        add_categories=True,
        nz_prop=0.1,
        use_raw=False,
        verbose=True
    )
    
    # --- SAFETY CHECK FOR PLOTTING ---
    # 1. Identify which targets actually exist in this sample's results
    #    (lrdata results are usually stored in .var_names)
    available_interactions = [
        i for i in target_interactions 
        if i in lrdata.var_names
    ]
    
    # 2. Only plot if we found at least one match
    if available_interactions:
        print(f"Plotting interactions: {available_interactions}")
        sc.pl.spatial(
            lrdata, 
            color=available_interactions, 
            size=1.4, 
            vmax=1, 
            cmap='coolwarm'
        )
    else:
        print(f"--> Skipping plot for {current_sample}: Target interactions not found.")

    # ... (Your existing code for appending results) ...
    current_int_df = lrdata.var.copy()  # Good practice to copy
    current_int_df['readout_id'] = current_sample
    current_int_df['condition'] = current_condition

    all_interactions.append(current_int_df)

In [ ]:
all_interactions_df = pd.concat(all_interactions)

In [ ]:
all_interactions_df

In [ ]:
susbet_df = all_interactions_df.loc[all_interactions_df.index.isin(target_interactions)]

In [ ]:
susbet_df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import pandas as pd

# 1. Setup the figure style for publication
sns.set_context("paper", font_scale=1.5) # 'paper' or 'talk' are good for pubs
sns.set_style("ticks") # Clean white background with ticks

# Create the figure
fig, ax = plt.subplots(figsize=(5, 6))

# 2. Define colors (optional, but good for distinction)
my_pal = {"Untreated": "#808080", "FAP_LTBR": "#E64B35"} # Grey vs Red

# 3. Create the Boxplot (Summary)
#    showfliers=False prevents outlier dots from appearing twice (since we add stripplot)
sns.boxplot(
    data=susbet_df, 
    x="condition", 
    y="mean", 
    palette=my_pal, 
    width=0.5, 
    linewidth=1.5,
    showfliers=False, 
    ax=ax
)

# 4. Add the Stripplot (Individual Points)
sns.stripplot(
    data=susbet_df, 
    x="condition", 
    y="mean", 
    color=".15", # Dark grey points
    size=8, 
    jitter=True, 
    alpha=0.7,
    ax=ax
)

# 5. Calculate Statistics (T-test or Mann-Whitney)
#    Split the data
group1 = susbet_df[susbet_df['condition'] == 'Untreated']['mean']
group2 = susbet_df[susbet_df['condition'] == 'FAP_LTBR']['mean']

#    Perform t-test (use stats.mannwhitneyu if non-normal)
t_stat, p_val = stats.ttest_ind(group1, group2)

# 6. Annotate the P-value manually
#    Get the y-axis limit to place the bar above the highest point
y_max = susbet_df['mean'].max()
y_line = y_max + 0.05 * y_max  # 5% higher than max point
h = 0.02 * y_max  # Height of the bracket legs

#    Draw the bracket
x1, x2 = 0, 1  # Coordinates for 'Untreated' and 'FAP_LTBR' (0 and 1 on x-axis)
plt.plot([x1, x1, x2, x2], [y_line, y_line+h, y_line+h, y_line], lw=1.5, c='k')

#    Add the text
significance = "ns"
if p_val < 0.001: significance = "***"
elif p_val < 0.01: significance = "**"
elif p_val < 0.05: significance = "*"

plt.text((x1+x2)*.5, y_line+h, f"{significance}\n(p={p_val:.4f})", 
         ha='center', va='bottom', color='k', fontsize=12)


# 7. Final Polish
ax.set_title("Interaction Strength: Ltb^Ltbr", fontweight='bold', pad=20)
ax.set_ylabel("Mean Interaction Score")
ax.set_xlabel("") # Remove x-label if categories are self-explanatory

#    Despine (remove top and right borders)
sns.despine(offset=10, trim=True)

plt.tight_layout()
plt.show()

# Optional: Save it
# plt.savefig("Ltb_Ltbr_comparison.pdf", dpi=300, bbox_inches='tight')

In [ ]:
# 1. Setup the figure style for publication
sns.set_context("paper", font_scale=1.5) # 'paper' or 'talk' are good for pubs
sns.set_style("ticks") # Clean white background with ticks

# Create the figure
fig, ax = plt.subplots(figsize=(5, 6))

# 2. Define colors (optional, but good for distinction)
my_pal = {"Untreated": "#808080", "FAP_LTBR": "#E64B35"} # Grey vs Red

# 3. Create the Boxplot (Summary)
#    showfliers=False prevents outlier dots from appearing twice (since we add stripplot)
sns.boxplot(
    data=susbet_df, 
    x="condition", 
    y="morans", 
    palette=my_pal, 
    width=0.5, 
    linewidth=1.5,
    showfliers=False, 
    ax=ax
)

# 4. Add the Stripplot (Individual Points)
sns.stripplot(
    data=susbet_df, 
    x="condition", 
    y="morans", 
    color=".15", # Dark grey points
    size=8, 
    jitter=True, 
    alpha=0.7,
    ax=ax
)

# 5. Calculate Statistics (T-test or Mann-Whitney)
#    Split the data
group1 = susbet_df[susbet_df['condition'] == 'Untreated']['morans']
group2 = susbet_df[susbet_df['condition'] == 'FAP_LTBR']['morans']

#    Perform t-test (use stats.mannwhitneyu if non-normal)
t_stat, p_val = stats.ttest_ind(group1, group2)

# 6. Annotate the P-value manually
#    Get the y-axis limit to place the bar above the highest point
y_max = susbet_df['morans'].max()
y_line = y_max + 0.05 * y_max  # 5% higher than max point
h = 0.02 * y_max  # Height of the bracket legs

#    Draw the bracket
x1, x2 = 0, 1  # Coordinates for 'Untreated' and 'FAP_LTBR' (0 and 1 on x-axis)
plt.plot([x1, x1, x2, x2], [y_line, y_line+h, y_line+h, y_line], lw=1.5, c='k')

#    Add the text
significance = "ns"
if p_val < 0.001: significance = "***"
elif p_val < 0.01: significance = "**"
elif p_val < 0.05: significance = "*"

plt.text((x1+x2)*.5, y_line+h, f"{significance}\n(p={p_val:.4f})", 
         ha='center', va='bottom', color='k', fontsize=12)


# 7. Final Polish
ax.set_title("Interaction Spatial Autocorrelation: Ltb^Ltbr", fontweight='bold', pad=20)
ax.set_ylabel("Mean Moran's Index Score")
ax.set_xlabel("") # Remove x-label if categories are self-explanatory

#    Despine (remove top and right borders)
sns.despine(offset=10, trim=True)

plt.tight_layout()
plt.show()

# Optional: Save it
# plt.savefig("Ltb_Ltbr_comparison.pdf", dpi=300, bbox_inches='tight')

In [ ]:
! jupyter nbconvert --to html 13_TargetExpressionExploration.ipynb